# Step 2: Generate data samples

In this section, we will use the amplitude model created above to generate a data sample via hit & miss Monte Carlo.

## 2.1 Generate

Using this amplitude model in the C++ side of ComPWA is simple. The `create_helicity_kinematics()` helper function builds a ``Kinematics`` object following the specifications in the XML model file. The second argument of this function is a particle list, which can be created with the `read_particles()` function. The created ``Kinematics`` object can be used to generate a phase space sample as well.

An ``Intensity`` object can be created with the `create_intensity()` function. Because the Intensities constructed by the Builder are automatically normalized, you have to pass this function a phase space sample as an argument. The ``Kinematics`` instance is then updated with the needed subsystems of the decay topology during the creation of the ``Intensity`` object.

Now all building blocks for generating our data sample are at hand.

In [ ]:
import pycompwa.ui as pwa

particle_list = pwa.read_particles('model.xml')

kinematics = pwa.create_helicity_kinematics('model.xml', particle_list)

generator = pwa.EvtGenGenerator(
    kinematics.get_particle_state_transition_kinematics_info())

random_generator = pwa.StdUniformRealGenerator(12345)

phsp_sample = pwa.generate_phsp(100000, generator, random_generator)

intensity_builder = pwa.IntensityBuilderXML(
    'model.xml', particle_list, kinematics, phsp_sample)

intensity = intensity_builder.create_intensity()

data_sample = pwa.generate(
    10000, kinematics, generator, intensity, random_generator)

## 2.2 Data Input/Output

You may want to write the generated data and phase space samples to disk, so that you don't have to perform this step each time you revisit the fit. ComPWA provides functions for ROOT and ASCII files, that can be used as follows:

In [ ]:
# to ROOT
pwa.write_root_data(data_sample, output_file='generated_data.root')
pwa.write_root_data(phsp_sample, output_file='generated_phsp.root')
# to ASCII
pwa.write_ascii_data(data_sample, output_file='generated_data.dat')
pwa.write_ascii_data(phsp_sample, output_file='generated_phsp.dat')

Importing syntax parallels that:

In [ ]:
imported_data = pwa.read_root_data(input_file='generated_data.root')
print('Imported events:', len(imported_data.events))
first_event = imported_data.events[0]
print('First event:')
print(first_event.four_momenta)
print(first_event.weight)

Note that the ASCII (`.dat`) files contain a header section that informs you about the final state of the file. You will have to prepend this section to the file yourself if you want to use a data sample from another framework.

In [ ]:
with open('generated_data.dat') as f:
    for _ in range(5):
        print(f.readline()[:-1])